# Working with Tabular Data

Lesson 12 - CSV files, load, read csv, manipulate within dictionaries stored as elements in a list, filter data based on criteria, leverage LLMs to suggest trip activities using extracted data.

Let's learn about csv files that structure data into rows and columns (tabular data yes!).

Text files are great but sometimes you need a bit more organization and structure, that's where csv files come into play.

Imagine you have a bunch of information about customer tickets organized in a .csv file that you would like to understand a bit more about.

In [3]:
# Super popular library for working with tabular data
import pandas as pd
from ai_tools import ask_ai

What is a .csv file?



In [2]:
import pandas as pd

data_customer_tickets = pd.read_csv("./extracted_ticket_issues.csv")

data_customer_tickets

customer_name                                  issue_description priority
0       Jane Doe  Customer was charged twice for the same transa...     High
1     John Smith  Customer unable to log into their account, fac...   Medium
2  Alice Johnson  Customer wants more information about product ...      Low
3      Bob Brown  Customer has not received the order yet, track...     High
4    Michael Lee  Customer wants to return a product and needs a...   Medium

The data contains 3 columns:
1. `customer_name` - names of the customers
2. `issue_description` - description of the issue they had
3. `priority` - reference to the level of priority of that task

We could use Python to get for example only the high priority issues:

In [4]:
# == indicates equivalence!
data_customer_tickets["priority"]=="High"

0     True
1    False
2    False
3     True
4    False
Name: priority, dtype: bool

In [5]:
high_priority_issues = data_customer_tickets[data_customer_tickets["priority"]=="High"]

Now we can take a look at the issues themselves:

In [6]:
high_priority_issues

customer_name                                  issue_description priority
0      Jane Doe  Customer was charged twice for the same transa...     High
3     Bob Brown  Customer has not received the order yet, track...     High

Awesome! What we could do now is for example use our `ask_ai` tool to categorize the issues for us to help organizing the information, and then feed that back into the table:

In [7]:
from ai_tools import ask_ai

ask_ai("Hi! I am teaching a course about automations with Python and some AI tools!")

'That sounds like an exciting course! There are so many applications for automation with Python and AI tools. Here are some ideas and topics you might consider covering in your course:\n\n### Python Automation\n1. **Basics of Python**:\n   - Data types, variables, control structures (loops, conditionals).\n   - Functions and modules.\n\n2. **Web Scraping**:\n   - Using libraries like BeautifulSoup and Scrapy to extract data from websites.\n   - Understanding ethical scraping and web scraping best practices.\n\n3. **File Handling**:\n   - Automating file management tasks (reading/writing files, organizing files).\n   - Working with CSV, JSON, and Excel files using pandas.\n\n4. **Task Automation with Scripts**:\n   - Scheduling tasks with `cron` (Linux) or Task Scheduler (Windows).\n   - Automating repetitive tasks, such as sending emails or generating reports.\n\n5. **APIs and Automation**:\n   - Making HTTP requests with `requests` and accessing RESTful APIs.\n   - Automating interact

In [8]:
categories_list = []
for issue in high_priority_issues["issue_description"]:
    print(f"Categorizing issue: {issue}")
    category = ask_ai(f"Categorize this issue in just one single word and OUTPUT ONLY THAT WORD:\n\n issue: {issue}\n category: \n")
    print(f"Category: {category}")
    categories_list.append(category)


Categorizing issue: Customer was charged twice for the same transaction.
Category: Billing
Categorizing issue: Customer has not received the order yet, tracking information shows a delay.
Category: Delay


Notice we use concepts we've learned before by looping over the issues, saving them to a list.

Now with that information in hand we can actually update the dataframe accordingly, first we create a new column in the dataframe:

In [9]:
data_customer_tickets["issue_category"] = None

In [10]:
# Update categories for high priority issues using the index from high_priority_issues
for idx, category in zip(high_priority_issues.index, categories_list):
    data_customer_tickets.loc[idx, "issue_category"] = category

In [11]:
data_customer_tickets

customer_name                                  issue_description priority  \
0       Jane Doe  Customer was charged twice for the same transa...     High   
1     John Smith  Customer unable to log into their account, fac...   Medium   
2  Alice Johnson  Customer wants more information about product ...      Low   
3      Bob Brown  Customer has not received the order yet, track...     High   
4    Michael Lee  Customer wants to return a product and needs a...   Medium   

  issue_category  
0        Billing  
1           None  
2           None  
3          Delay  
4           None

Notice that the issues for which we did not analyse still contain a `None` indicating they haven't been categorized yet!

## Creating and Managing Structured Data

Besides analysing data, we can also create our ownn tables with information we care about.

Let's start with a practical example - creating a camping trip gear checklist:

In [12]:
# Create a camping gear checklist
camping_gear = {
    "item": [
        "Tent", "Sleeping Bag", "Backpack", "Hiking Boots",
        "Water Filter", "First Aid Kit", "Headlamp", "Camp Stove"
    ],
    "priority": [
        "Essential", "Essential", "Essential", "Essential",
        "High", "Essential", "High", "Medium"
    ],
    "estimated_cost": [
        299.99, 149.99, 199.99, 159.99,
        89.99, 49.99, 39.99, 79.99
    ],
    "packed": [
        False, False, False, False,
        False, False, False, False
    ]
}

# Convert to DataFrame
gear_df = pd.DataFrame(camping_gear)
print("Camping Gear Checklist:")
display(gear_df)

Camping Gear Checklist:


item   priority  estimated_cost  packed
0           Tent  Essential          299.99   False
1   Sleeping Bag  Essential          149.99   False
2       Backpack  Essential          199.99   False
3   Hiking Boots  Essential          159.99   False
4   Water Filter       High           89.99   False
5  First Aid Kit  Essential           49.99   False
6       Headlamp       High           39.99   False
7     Camp Stove     Medium           79.99   False

## Working with Data Filters

Let's demonstrate how to filter and analyze our data:

In [13]:
def analyze_gear_requirements():
    # Filter essential items
    essential_gear = gear_df[gear_df['priority'] == 'Essential']
    
    # Calculate total cost of essential items
    essential_cost = essential_gear['estimated_cost'].sum()
    
    # Get unpacked essential items
    unpacked_essential = essential_gear[~essential_gear['packed']]
    
    print(f"Total cost of essential gear: ${essential_cost:.2f}")
    print("\nUnpacked essential items:")
    display(unpacked_essential[['item', 'estimated_cost']])

analyze_gear_requirements()

Total cost of essential gear: $859.95

Unpacked essential items:


item  estimated_cost
0           Tent          299.99
1   Sleeping Bag          149.99
2       Backpack          199.99
3   Hiking Boots          159.99
5  First Aid Kit           49.99

## Creating a Trip Itinerary

Let's create a more complex example with a detailed trip itinerary:

In [14]:
def create_trip_itinerary():
    itinerary_data = {
        'day': range(1, 6),
        'date': pd.date_range('2024-06-01', periods=5),
        'activity': [
            'Arrival and Camp Setup',
            'Mountain Trail Hike',
            'Lake Exploration',
            'Forest Adventure',
            'Pack and Departure'
        ],
        'location': [
            'Basecamp Area',
            'Mountain Ridge Trail',
            'Crystal Lake',
            'Ancient Forest',
            'Basecamp Area'
        ],
        'distance_km': [2, 8, 5, 6, 2],
        'difficulty': [
            'Easy',
            'Hard',
            'Moderate',
            'Moderate',
            'Easy'
        ]
    }
    
    itinerary_df = pd.DataFrame(itinerary_data)
    return itinerary_df

# Create and display the itinerary
trip_itinerary = create_trip_itinerary()
print("Trip Itinerary:")
display(trip_itinerary)

Trip Itinerary:


day       date                activity              location  distance_km  \
0    1 2024-06-01  Arrival and Camp Setup         Basecamp Area            2   
1    2 2024-06-02     Mountain Trail Hike  Mountain Ridge Trail            8   
2    3 2024-06-03        Lake Exploration          Crystal Lake            5   
3    4 2024-06-04        Forest Adventure        Ancient Forest            6   
4    5 2024-06-05      Pack and Departure         Basecamp Area            2   

  difficulty  
0       Easy  
1       Hard  
2   Moderate  
3   Moderate  
4       Easy

## Analyzing Trip Statistics

Let's add some analysis to our trip planning:

In [15]:
def analyze_trip_metrics(itinerary_df):
    # Calculate total distance
    total_distance = itinerary_df['distance_km'].sum()
    
    # Get difficulty breakdown
    difficulty_counts = itinerary_df['difficulty'].value_counts()
    
    # Find longest day
    longest_day = itinerary_df.loc[itinerary_df['distance_km'].idxmax()]
    
    print(f"Trip Analysis:")
    print(f"Total distance: {total_distance} km")
    print("\nDifficulty breakdown:")
    display(difficulty_counts)
    print(f"\nLongest day: Day {longest_day['day']} - {longest_day['activity']}")
    print(f"Distance: {longest_day['distance_km']} km")

analyze_trip_metrics(trip_itinerary)

Trip Analysis:
Total distance: 23 km

Difficulty breakdown:


difficulty
Easy        2
Moderate    2
Hard        1
Name: count, dtype: int64


Longest day: Day 2 - Mountain Trail Hike
Distance: 8 km


## Exporting and Saving Data

Let's see how to save our data for later use:

In [16]:
def export_trip_data(gear_df, itinerary_df, filename_prefix):
    # Export to CSV
    gear_df.to_csv(f"{filename_prefix}_gear.csv", index=False)
    itinerary_df.to_csv(f"{filename_prefix}_itinerary.csv", index=False)

# Export our data
export_trip_data(gear_df, trip_itinerary, "camping_trip")

## Practical Exercise: Trip Budget Calculator

Let's create a budget calculator for our trip:

In [ ]:
def calculate_trip_budget(gear_df, itinerary_df):
    # Equipment costs
    total_gear_cost = gear_df['estimated_cost'].sum()
    
    # Daily expenses (example values)
    daily_expenses = {
        'food': 30,
        'fuel': 10,
        'miscellaneous': 15
    }
    
    num_days = len(itinerary_df)
    daily_total = sum(daily_expenses.values())
    total_daily_costs = daily_total * num_days
    
    # Create budget summary
    budget_summary = pd.DataFrame({
        'Category': ['Gear', 'Food', 'Fuel', 'Miscellaneous'],
        'Cost': [
            total_gear_cost,
            daily_expenses['food'] * num_days,
            daily_expenses['fuel'] * num_days,
            daily_expenses['miscellaneous'] * num_days
        ]
    })
    
    budget_summary['Percentage'] = (
        budget_summary['Cost'] / budget_summary['Cost'].sum() * 100
    ).round(1)
    
    return budget_summary

# Calculate and display budget
budget = calculate_trip_budget(gear_df, trip_itinerary)
print("Trip Budget Summary:")
display(budget)

Trip Budget Summary:


Category     Cost  Percentage
0           Gear  1069.92        79.6
1           Food   150.00        11.2
2           Fuel    50.00         3.7
3  Miscellaneous    75.00         5.6

## Key Takeaways

- Pandas provides powerful tools for working with tabular data
- DataFrames can be filtered and analyzed in various ways
- Data can be exported to different formats (CSV, Excel)
- Structured data makes analysis and planning easier
- Always consider data types when creating DataFrames
- Use appropriate column names and data organization
- Remember to handle missing data appropriately

In the next lesson, we'll explore more advanced pandas operations and data visualization techniques!